# 단어의 표현 (Word Representation)


기계는 문자를 그대로 인식할 수 없기때문에 숫자로 변환



# 1 원-핫 인코딩 (One-Hot Encoding)

## 1.1 직접 구현해보기

### "원숭이, 바나나, 사과" 로 원-핫 인코딩을 한다면

In [ ]:
# 인코딩 대상 단어들을 담은 리스트
word_ls = ['원숭이','바나나','사과']

In [ ]:
# 원핫 인코딩 생성


In [ ]:
one_hot_vectors = one_hot_encode(word_ls)
one_hot_vectors

### "코끼리"라는 단어가 추가된다면?

In [ ]:
word_ls = ['원숭이','바나나','사과','코끼리']

In [ ]:
one_hot_vectors = one_hot_encode(word_ls)
one_hot_vectors

In [ ]:
# 라벨인코딩 ?

## 1.3 sklearn 활용

- LabelEncoder() : 유니크한 범주형 값들을 1부터 증가하는 숫자로 인코딩
- OneHotEncoder() : 유니크한 범주형 값들을 n:1 행렬로 변환

함수명 | 설명
--|--
fit(X[, y])	| Fit OneHotEncoder to X.
fit_transform(X[, y])	| Fit OneHotEncoder to X, then transform X.
inverse_transform(X)	| Convert the back data to the original representation.
transform(X)	| Transform X using one-hot encoding.

In [ ]:
# sklearn을 활용한 one-hot encoding
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# 예제 데이터 배열
values = array(word_ls)
print(values)

# 문자열에 숫자를 붙임
label_enc = LabelEncoder()
int_enc = label_enc.fit_transform(values)
print(int_enc)

# binary encode
onehot_enc = OneHotEncoder(sparse=False)
int_enc = int_enc.reshape(len(int_enc), 1) # n:1 matrix로 변환
print(int_enc)
onehot_enc = onehot_enc.fit_transform(int_enc)
print(onehot_enc)

# one-hot encoding 의 첫번째 배열을 값을 역으로 산출
inverted = label_enc.inverse_transform([argmax(onehot_enc[0, :])])
print(inverted)



---



# 2 밀집 벡터 (Dense Vector)

## 2-1 유사도 계산

### 2.1.1 유클리디언 거리(Euclidean distance)
두 벡터사이의 직선 거리. 피타고라스 정리를 생각하면 이해하기 쉬움

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/55/Euclidean_distance_2d.svg/220px-Euclidean_distance_2d.svg.png"  width="200"/>

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/795b967db2917cdde7c2da2d1ee327eb673276c0" width="350"/>

https://en.wikipedia.org/wiki/Euclidean_distance

In [ ]:
word_embedding_dic = {
    '사과' : [1.0, 0.5],
    '바나나' : [0.9, 1.2],
    '원숭이' : [0.5, 1.5]
}

In [ ]:
import numpy as np
# 유클리디언 거리 계산


# 사과와 바나나의 코사인 유사도
euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나'])

### 2.1.2 자카드 유사도(Jaccard index)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Intersection_of_sets_A_and_B.svg/200px-Intersection_of_sets_A_and_B.svg.png" />

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/eaef5aa86949f49e7dc6b9c8c3dd8b233332c9e7" />

https://en.wikipedia.org/wiki/Jaccard_index

In [ ]:
s1 = '대부분 원숭이는 바나나를 좋아합니다.'
s2 = '코주부 원숭이는 바나나를 싫어합니다.'

# 토큰화를 수행합니다.
token_s1 = s1.split()
token_s2 = s2.split()

# 자카드 유사도 계산


### 2.1.3 코사인 유사도(Cosine Similarity)

*  두 벡터간의 유사도를 측정하는 방법 중 하나
*  두 벡터 사이의 코사인을 측정
*  0도 = 1, 90도 = 0, 180도 = -1   ==> 1에 가까울수록 유사도가 높음




<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/1d94e5903f7936d3c131e040ef2c51b473dd071d" width='350'/>

https://en.wikipedia.org/wiki/Cosine_similarity

In [ ]:
# 코사인 유사도 계산


In [ ]:
# 사과와 바나나의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['바나나']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나']))

In [ ]:

# 사과와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['원숭이']))

In [ ]:
# 바나나와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))



---



# 3 TF-IDF를 활용한 단어 벡터

### 3.1 직접 구현하기

weighting schema|weight|설명
--|--|--
term frequency|<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/91699003abf4fe8bdf861bbce08e73e71acf5fd4" />|=토큰빈도/문서내토큰빈도
inverse document frequency|<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/864fcfdc0c16344c11509f724f1aa7081cf9f657" />|=log(총문서갯수/(토큰이 등장한 문서수))

In [ ]:
d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"

In [ ]:
corpus = [d1, d2]

In [ ]:
import numpy as np

# TF-IDF 계산

# document 내 token 등장비율 TF

# 단어 - 문서 등장 횟수 IDF

# 두 값의 곱


### 3.2 sklearn 활용

- feature_extraction.text.CountVectorizer() : 문장들에서 단어의 빈도수를 벡터화

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]
count_vect = CountVectorizer()
countv = count_vect.fit_transform(corpus)
print(countv.toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(count_vect.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

- feature_extraction.text.TfidfVectorizer() : 문장들을 TF-IDF로 벡터화

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]
tfidf_vect = TfidfVectorizer().fit(corpus)
tfidfv = tfidf_vect.transform(corpus)
print(tfidfv.toarray())
print(tfidf_vect.vocabulary_)

## 3.3 gensim 활용

- corpora.Dictionary - 토큰화한 문서들 내에서 index-단어 사전을 생성
- models.TfidfModel() - TF-IDF 벡터화

In [ ]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim import corpora

d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]

doc_ls = [doc.split() for doc in corpus]
id2word = corpora.Dictionary(doc_ls)  # fit dictionary
corpus = [id2word.doc2bow(doc) for doc in doc_ls]  # convert corpus to BoW format

tfidf = TfidfModel(corpus)  # fit model
vector = tfidf[corpus[0]]  # apply model to the first corpus document

In [ ]:
tfidf[corpus][0]



---



# 4 LSA(Latent Semantic Analysis)를 활용한 단어 벡터

## 4.1 sklearn 활용

- decomposition.TruncatedSVD() : 특이값분해

In [ ]:
doc_ls = [
    '바나나 사과 포도 포도',
    '사과 포도',
    '포도 바나나',
    '짜장면 짬뽕 탕수육',
    '볶음밥 탕수육',
    '짜장면 짬뽕',
    '라면 스시',
    '스시',
    '가츠동 스시 소바',
    '된장찌개 김치찌개 김치',
    '김치 된장',
    '비빔밥 김치'
]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

count_vect = CountVectorizer()
countv = count_vect.fit_transform(doc_ls)
svd = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100)
svd.fit(countv)

features = count_vect.get_feature_names_out() # 단어 집합. 1,000개의 단어가 저장됨.
for i in range(len(features)) :
    print("{} : {}".format(features[i], svd.components_[:,i]))

## 4.2 gensim 활용

- models.LsiModel() : LSA 모델

In [ ]:
doc_ls = [
    '바나나 사과 포도 포도',
    '사과 포도',
    '포도 바나나',
    '짜장면 짬뽕 탕수육',
    '볶음밥 탕수육',
    '짜장면 짬뽕',
    '라면 스시',
    '스시',
    '가츠동 스시 소바',
    '된장찌개 김치찌개 김치',
    '김치 된장',
    '비빔밥 김치'
]
doc_ls = [d.split() for d in doc_ls]

In [ ]:
from gensim import corpora
from gensim.models import LsiModel

id2word = corpora.Dictionary(doc_ls) #사전 구축
corpus = [id2word.doc2bow(text) for text in doc_ls] # 코퍼스 생성
lsi = LsiModel(corpus, id2word=id2word, num_topics=2) #LSA 모델

for i in id2word.keys() :
    print("{} : {}".format(id2word[i], lsi.projection.u[i]))